In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U "transformers>=4.42.0" "accelerate>=0.26.0" einops timm

## Local Inference on GPU
Model page: https://huggingface.co/microsoft/Florence-2-large

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/microsoft/Florence-2-large)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor

# 1. 필수 라이브러리 체크 (버전 충돌 방지)
# !pip install -U transformers einops timm

model_id = "microsoft/Florence-2-large"
device = "cuda" if torch.cuda.is_available() else "cpu"

# 2. 모델 로드 (에러 유발 인자를 제거하고 최소한으로 로드)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    attn_implementation="eager"  # SDPA 에러 방지
).to(device)

# 메모리 절약을 위해 필요한 경우에만 별도로 half() 변환
# model = model.half()

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

print("✅ 모델 로드 성공!")

In [ ]:
def run_florence2(image, task_prompt="<DETAILED_CAPTION>"):
    # 1. 이미지 전처리
    inputs = processor(text=task_prompt, images=image, return_tensors="pt").to(device)

    # 2. 텍스트 생성 (에러 수정 포인트)
    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=1024,
            num_beams=3,
            do_sample=False,
            # [수정] 아래 두 옵션이 에러를 방지합니다.
            use_cache=False,
            early_stopping=True
        )

    # 3. 결과 디코딩 및 후처리
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )

    return parsed_answer[task_prompt]

In [ ]:
# 이미지 로드 (업로드한 파일 경로 혹은 URL)
image_path = "/content/스크린샷 2025-12-28 오후 5.14.07.png"
img = Image.open(image_path).convert("RGB")

# 테스트 실행
# 1. 상세 묘사
caption = run_florence2(img, task_prompt="<DETAILED_CAPTION>")
print(f"상세 묘사: {caption}")

# # # 2. 대사 추출 (OCR) - 웹툰의 문맥 파악에 중요합니다.
# ocr_text = run_florence2(img, task_prompt="<OCR>")
# print(f"추출된 대사: {ocr_text}")